# Interpolation Methods Showcase

This notebook demonstrates the various interpolation methods available in rustlab-numerical, including linear, polynomial, and spline interpolation techniques.

## Setup and Dependencies

In [ ]:
:dep rustlab-math = { path = "../../rustlab-math" }
:dep rustlab-numerical = { path = ".." }
:dep rustlab-plotting = { path = "../../rustlab-plotting" }

use rustlab_math::{VectorF64, ArrayF64, vec64, array64, range, matrix};
use rustlab_numerical::interpolation::*;
use rustlab_plotting::*;
use std::f64::consts::PI;

## 1. Linear Interpolation

Linear interpolation is the simplest and fastest method, providing O(log n) evaluation time.

In [3]:
{
    // Create sample data points using ergonomic macros
    let x_data = vec64![0.0, 1.0, 2.0, 3.0, 4.0, 5.0];
    let y_data = vec64![0.0, 1.0, 4.0, 9.0, 16.0, 25.0]; // x^2

    println!("Original data points:");
    for (i, (&x, &y)) in x_data.iter().zip(y_data.iter()).enumerate() {
        println!("{}", format!("Point {}: ({:.1}, {:.1})", i, x, y));
    }
}

Original data points:
Point 0: (0.0, 0.0)
Point 1: (1.0, 1.0)
Point 2: (2.0, 4.0)
Point 3: (3.0, 9.0)
Point 4: (4.0, 16.0)
Point 5: (5.0, 25.0)


()

In [5]:
{
    // Create linear interpolator and test at intermediate points
    let x_data = vec64![0.0, 1.0, 2.0, 3.0, 4.0, 5.0];
    let y_data = vec64![0.0, 1.0, 4.0, 9.0, 16.0, 25.0];
    let linear_interp = LinearInterpolator::new(&x_data, &y_data).unwrap();

    let test_points = vec![0.5, 1.5, 2.5, 3.5, 4.5];
    println!("Linear interpolation results:");
    for x in test_points {
        let y_interp = linear_interp.eval(x).unwrap();
        let y_exact = x * x; // True function
        let error = (y_interp - y_exact).abs();
        println!("{}", format!("x = {:.1}: interpolated = {:.3}, exact = {:.3}, error = {:.3}", 
                 x, y_interp, y_exact, error));
    }
}

Error: arguments to this function are incorrect

## 2. Polynomial Interpolation

Polynomial interpolation provides exact fitting through all data points but can suffer from Runge's phenomenon.

In [6]:
{
    // Use fewer points to avoid oscillations
    let x_poly = vec64![0.0, 1.0, 2.0, 3.0];
    let y_poly = vec64![0.0, 1.0, 4.0, 9.0];
    let lagrange_interp = LagrangeInterpolator::new(&x_poly, &y_poly).unwrap();

    let test_points = vec![0.5, 1.5, 2.5];
    println!("Polynomial interpolation results:");
    for x in test_points {
        let y_interp = lagrange_interp.eval(x).unwrap();
        let y_exact = x * x;
        let error = (y_interp - y_exact).abs();
        println!("{}", format!("x = {:.1}: interpolated = {:.6}, exact = {:.6}, error = {:.2e}", 
                 x, y_interp, y_exact, error));
    }
}

Error: arguments to this function are incorrect

## 3. Cubic Spline Interpolation

Cubic splines provide smooth interpolation with C² continuity and avoid oscillations.

In [7]:
{
    // Create natural cubic spline with correct API
    let x_data = vec64![0.0, 1.0, 2.0, 3.0, 4.0, 5.0];
    let y_data = vec64![0.0, 1.0, 4.0, 9.0, 16.0, 25.0];
    let spline = CubicSpline::new(x_data.clone(), y_data.clone(), BoundaryCondition::Natural).unwrap();

    let test_points = vec![0.5, 1.5, 2.5, 3.5, 4.5];
    println!("Cubic spline interpolation results:");
    for x in test_points {
        let y_interp = spline.eval(x).unwrap();
        let y_exact = x * x;
        let error = (y_interp - y_exact).abs();
        println!("{}", format!("x = {:.1}: interpolated = {:.6}, exact = {:.6}, error = {:.3}", 
                 x, y_interp, y_exact, error));
    }
}

Cubic spline interpolation results:
x = 0.5: interpolated = 0.342105, exact = 0.250000, error = 0.092
x = 1.5: interpolated = 2.223684, exact = 2.250000, error = 0.026
x = 2.5: interpolated = 6.263158, exact = 6.250000, error = 0.013
x = 3.5: interpolated = 12.223684, exact = 12.250000, error = 0.026
x = 4.5: interpolated = 20.342105, exact = 20.250000, error = 0.092


()

In [8]:
{
    // Test spline derivative evaluation with correct API
    let x_data = vec64![0.0, 1.0, 2.0, 3.0, 4.0, 5.0];
    let y_data = vec64![0.0, 1.0, 4.0, 9.0, 16.0, 25.0];
    let spline = CubicSpline::new(x_data.clone(), y_data.clone(), BoundaryCondition::Natural).unwrap();

    let test_points = vec![0.5, 1.5, 2.5, 3.5, 4.5];
    println!("Spline derivative evaluation:");
    for x in test_points {
        let dy_dx = spline.eval_derivative(x).unwrap();
        let exact_derivative = 2.0 * x; // d/dx(x²) = 2x
        let error = (dy_dx - exact_derivative).abs();
        println!("{}", format!("x = {:.1}: spline derivative = {:.3}, exact = {:.3}, error = {:.3}", 
                 x, dy_dx, exact_derivative, error));
    }
}

Spline derivative evaluation:
x = 0.5: spline derivative = 0.895, exact = 1.000, error = 0.105
x = 1.5: spline derivative = 3.026, exact = 3.000, error = 0.026
x = 2.5: spline derivative = 5.000, exact = 5.000, error = 0.000
x = 3.5: spline derivative = 6.974, exact = 7.000, error = 0.026
x = 4.5: spline derivative = 9.105, exact = 9.000, error = 0.105


()

## 4. Visualization Comparison

In [9]:
{
    // Generate dense evaluation points and create interpolators
    let x_data = vec64![0.0, 1.0, 2.0, 3.0, 4.0, 5.0];
    let y_data = vec64![0.0, 1.0, 4.0, 9.0, 16.0, 25.0];
    let linear_interp = LinearInterpolator::new(&x_data, &y_data).unwrap();
    let spline = CubicSpline::new(x_data.clone(), y_data.clone(), BoundaryCondition::Natural).unwrap();
    
    let x_eval = range!(0.0 => 5.0, 101);
    let y_linear: VectorF64 = x_eval.iter().map(|&x| linear_interp.eval(x).unwrap_or(f64::NAN)).collect();
    let y_spline: VectorF64 = x_eval.iter().map(|&x| spline.eval(x).unwrap_or(f64::NAN)).collect();
    let y_exact: VectorF64 = x_eval.iter().map(|&x| x * x).collect();

    // Create comparison plot using correct plotting API
    Plot::new()
        .scatter_with(&x_data, &y_data, "Data Points")
        .line_with(&x_eval, &y_exact, "Exact (x²)")
        .line_with(&x_eval, &y_linear, "Linear")
        .line_with(&x_eval, &y_spline, "Cubic Spline")
        .title("Interpolation Methods Comparison")
        .xlabel("x")
        .ylabel("y")
        .legend(true)
        .grid(true)
        .show()?;
}

Error: arguments to this function are incorrect

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

## 5. Error Analysis

Let's analyze the interpolation errors quantitatively.

In [10]:
{
    // Calculate RMS errors with correct API
    let x_data = vec64![0.0, 1.0, 2.0, 3.0, 4.0, 5.0];
    let y_data = vec64![0.0, 1.0, 4.0, 9.0, 16.0, 25.0];
    let linear_interp = LinearInterpolator::new(&x_data, &y_data).unwrap();
    let spline = CubicSpline::new(x_data.clone(), y_data.clone(), BoundaryCondition::Natural).unwrap();
    
    let x_eval = range!(0.0 => 5.0, 101);
    let linear_errors: Vec<f64> = x_eval.iter()
        .filter_map(|&x| {
            if x >= 0.0 && x <= 5.0 {
                let exact = x * x;
                let linear_val = linear_interp.eval(x).ok()?;
                Some((linear_val - exact).abs())
            } else {
                None
            }
        })
        .collect();
    
    let spline_errors: Vec<f64> = x_eval.iter()
        .filter_map(|&x| {
            if x >= 0.0 && x <= 5.0 {
                let exact = x * x;
                let spline_val = spline.eval(x).ok()?;
                Some((spline_val - exact).abs())
            } else {
                None
            }
        })
        .collect();

    let linear_rms = (linear_errors.iter().map(|&e| e * e).sum::<f64>() / linear_errors.len() as f64).sqrt();
    let spline_rms = (spline_errors.iter().map(|&e| e * e).sum::<f64>() / spline_errors.len() as f64).sqrt();

    println!("RMS Errors:");
    println!("{}", format!("Linear interpolation: {:.6}", linear_rms));
    println!("{}", format!("Cubic spline: {:.6}", spline_rms));
    println!("{}", format!("Improvement factor: {:.2}x", linear_rms / spline_rms));
}

Error: arguments to this function are incorrect

## 6. 2D Interpolation Example

Demonstration of bilinear interpolation for 2D data.

In [11]:
{
    // Create 2D grid data using ergonomic macros
    let x_grid = vec64![0.0, 1.0, 2.0];
    let y_grid = vec64![0.0, 1.0, 2.0];

    // Function: f(x,y) = x² + y² - create with array64 macro for cleaner code
    let z_data = array64![
        [0.0, 1.0, 4.0],  // y=0: x²+0² for x=0,1,2
        [1.0, 2.0, 5.0],  // y=1: x²+1² for x=0,1,2  
        [4.0, 5.0, 8.0]   // y=2: x²+4² for x=0,1,2
    ];

    println!("2D Grid Data (z = x² + y²):");
    for i in 0..3 {
        let row: Vec<String> = (0..3).map(|j| format!("{:6.1}", z_data.get(i, j).unwrap())).collect();
        println!("{}", row.join(" "));
    }
}

2D Grid Data (z = x² + y²):
   0.0    1.0    4.0
   1.0    2.0    5.0
   4.0    5.0    8.0


()

In [12]:
{
    // Create bilinear interpolator and test
    let x_grid = vec64![0.0, 1.0, 2.0];
    let y_grid = vec64![0.0, 1.0, 2.0];
    let z_data = array64![
        [0.0, 1.0, 4.0],
        [1.0, 2.0, 5.0],
        [4.0, 5.0, 8.0]
    ];
    
    let bilinear = BilinearInterpolator::new(&x_grid, &y_grid, &z_data).unwrap();
    let test_2d_points = vec![(0.5, 0.5), (1.5, 0.5), (0.5, 1.5), (1.5, 1.5)];

    println!("Bilinear interpolation results:");
    for (x, y) in test_2d_points {
        let z_interp = bilinear.eval(x, y).unwrap();
        let z_exact = x*x + y*y;
        let error = (z_interp - z_exact).abs();
        println!("{}", format!("({:.1}, {:.1}): interpolated = {:.3}, exact = {:.3}, error = {:.3}", 
                 x, y, z_interp, z_exact, error));
    }
}

Error: arguments to this function are incorrect

## 7. Performance Comparison

Let's benchmark the different interpolation methods.

In [13]:
{
    // Performance comparison with correct API
    use std::time::Instant;

    // Create larger dataset for benchmarking using range macro
    let n_points = 1000;
    let x_large = range!(0.0 => 10.0, n_points);
    let y_large: VectorF64 = x_large.iter().map(|&x| (x * PI).sin()).collect();

    // Create interpolators
    let linear_large = LinearInterpolator::new(&x_large, &y_large).unwrap();
    let spline_large = CubicSpline::new(x_large.clone(), y_large.clone(), BoundaryCondition::Natural).unwrap();

    // Benchmark evaluation
    let n_evals = 10000;
    let eval_points = range!(0.0 => 10.0, n_evals);

    println!("{}", format!("Performance benchmark ({} evaluations):", n_evals));

    // Linear interpolation benchmark
    let start = Instant::now();
    for x in eval_points.iter() {
        let _ = linear_large.eval(*x);
    }
    let linear_time = start.elapsed();

    // Spline interpolation benchmark  
    let start = Instant::now();
    for x in eval_points.iter() {
        let _ = spline_large.eval(*x);
    }
    let spline_time = start.elapsed();

    println!("{}", format!("Linear interpolation: {:.2?} ({:.1} ns/eval)", 
             linear_time, linear_time.as_nanos() as f64 / n_evals as f64));
    println!("{}", format!("Cubic spline: {:.2?} ({:.1} ns/eval)", 
             spline_time, spline_time.as_nanos() as f64 / n_evals as f64));
    println!("{}", format!("Speed ratio: {:.1}x", 
             spline_time.as_nanos() as f64 / linear_time.as_nanos() as f64));
}

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: arguments to this function are incorrect

## 8. Advanced Example: Signal Reconstruction

Practical example using interpolation for signal processing.

In [14]:
{
    // Create a signal with missing samples using range macro
    let t_full = range!(0.0 => 4.0 * PI, 200);
    let signal_full: VectorF64 = t_full.iter()
        .map(|&t| (t).sin() + 0.5 * (3.0 * t).sin() + 0.1 * (10.0 * t).cos())
        .collect();

    // Subsample the signal (simulate missing data)
    let step = 8;
    let t_sparse: VectorF64 = t_full.iter().step_by(step).copied().collect();
    let signal_sparse: VectorF64 = signal_full.iter().step_by(step).copied().collect();

    println!("{}", format!("Original signal: {} samples", t_full.len()));
    println!("{}", format!("Sparse signal: {} samples ({:.1}% of original)", 
             t_sparse.len(), 
             100.0 * t_sparse.len() as f64 / t_full.len() as f64));
}

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

In [17]:
{
    // Reconstruct signal using different methods
    let t_full = range!(0.0 => 4.0 * PI, 200);
    let signal_full: VectorF64 = t_full.iter()
        .map(|&t| (t).sin() + 0.5 * (3.0 * t).sin() + 0.1 * (10.0 * t).cos())
        .collect();
    
    let step = 8;
    let t_sparse: VectorF64 = t_full.iter().step_by(step).copied().collect();
    let signal_sparse: VectorF64 = signal_full.iter().step_by(step).copied().collect();
    
    let linear_signal = LinearInterpolator::new(&t_sparse, &signal_sparse).unwrap();
    let spline_signal = CubicSpline::new(t_sparse.clone(), signal_sparse.clone(), BoundaryCondition::Natural).unwrap();

    // Evaluate reconstructed signals
    let signal_linear: VectorF64 = t_full.iter()
        .map(|&t| linear_signal.eval(t).unwrap_or(0.0))
        .collect();
    let signal_spline: VectorF64 = t_full.iter()
        .map(|&t| spline_signal.eval(t).unwrap_or(0.0))
        .collect();

    // Calculate reconstruction errors
    let linear_error: f64 = signal_full.iter().zip(signal_linear.iter())
        .map(|(&orig, &recon)| (orig - recon).powi(2))
        .sum::<f64>().sqrt() / signal_full.len() as f64;

    let spline_error: f64 = signal_full.iter().zip(signal_spline.iter())
        .map(|(&orig, &recon)| (orig - recon).powi(2))
        .sum::<f64>().sqrt() / signal_full.len() as f64;

    println!("Reconstruction RMS errors:");
    println!("{}", format!("Linear: {:.6}", linear_error));
    println!("{}", format!("Spline: {:.6}", spline_error));
    println!("{}", format!("Improvement: {:.1}x", linear_error / spline_error));
}

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: arguments to this function are incorrect

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

In [16]:
{
    // Visualize signal reconstruction
    let t_full = range!(0.0 => 4.0 * PI, 200);
    let signal_full: VectorF64 = t_full.iter()
        .map(|&t| (t).sin() + 0.5 * (3.0 * t).sin() + 0.1 * (10.0 * t).cos())
        .collect();
    
    let step = 8;
    let t_sparse: VectorF64 = t_full.iter().step_by(step).copied().collect();
    let signal_sparse: VectorF64 = signal_full.iter().step_by(step).copied().collect();
    
    let linear_signal = LinearInterpolator::new(&t_sparse, &signal_sparse).unwrap();
    let spline_signal = CubicSpline::new(t_sparse.clone(), signal_sparse.clone(), BoundaryCondition::Natural).unwrap();
    
    let signal_linear: VectorF64 = t_full.iter()
        .map(|&t| linear_signal.eval(t).unwrap_or(0.0))
        .collect();
    let signal_spline: VectorF64 = t_full.iter()
        .map(|&t| spline_signal.eval(t).unwrap_or(0.0))
        .collect();

    Plot::new()
        .line_with(&t_full, &signal_full, "Original")
        .scatter_with(&t_sparse, &signal_sparse, "Samples")
        .line_with(&t_full, &signal_linear, "Linear Reconstruction")
        .line_with(&t_full, &signal_spline, "Spline Reconstruction")
        .title("Signal Reconstruction with Interpolation")
        .xlabel("Time")
        .ylabel("Amplitude")
        .legend(true)
        .grid(true)
        .show()?;
}

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: arguments to this function are incorrect

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

Error: a value of type `rustlab_math::Vector<f64>` cannot be built from an iterator over elements of type `f64`

## Summary

This notebook demonstrated:

1. **Linear Interpolation**: Fast O(log n) evaluation, simple implementation
2. **Polynomial Interpolation**: Exact fitting but prone to oscillations
3. **Cubic Splines**: Smooth C² interpolation with derivative support
4. **2D Interpolation**: Extension to multidimensional data
5. **Performance**: Trade-offs between accuracy and computational cost
6. **Applications**: Signal reconstruction and data analysis

### Key Takeaways:

- Choose linear interpolation for speed when accuracy requirements are modest
- Use cubic splines for smooth interpolation with good accuracy
- Avoid high-degree polynomial interpolation due to Runge's phenomenon
- Consider the trade-off between computational cost and accuracy for your specific application
- 2D interpolation extends naturally to higher dimensions with appropriate algorithms

### Next Steps:

- Explore advanced spline variants (B-splines, NURBS)
- Study adaptive interpolation strategies
- Investigate interpolation for scattered data
- Learn about interpolation error bounds and convergence theory